### Sentiment Analysis using Spark ML Pipeline

#### Objective
- Build a scalable text classification pipeline using PySpark, including:
- Text preprocessing
- Feature extraction (TF-IDF)
- Logistic Regression classification
- Model evaluation

#### Dataset Description

The dataset (sentiments.csv) contains two columns:
- `text`: textual content
- `sentiment`: sentiment label (-1 = negative, 1 = positive)

The labels are normalized to:
- 0 → negative
- 1 → positive

In [20]:
# import thư viện
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

from pyspark.ml import Pipeline
from pyspark.ml.feature import (
    Tokenizer,
    StopWordsRemover,
    HashingTF,
    IDF
)

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [21]:
# khởi tạo spark session
spark = (
    SparkSession.builder
    .appName("SentimentAnalysis")
    .getOrCreate()
)


In [22]:
# load dataset
data_path = r"C:\Users\DoubleDD\HUS\NLP&DL\datasets\sentiments.csv"

df = spark.read.csv(
    data_path,
    header=True,
    inferSchema=True
)

df.show(5)


+--------------------+---------+
|                text|sentiment|
+--------------------+---------+
|Kickers on my wat...|        1|
|user: AAP MOVIE. ...|        1|
|user I'd be afrai...|        1|
|   MNTA Over 12.00  |        1|
|    OI  Over 21.37  |        1|
+--------------------+---------+
only showing top 5 rows



In [23]:
# Convert sentiment labels from {-1, 1} to {0, 1}
df = df.withColumn(
    "label",
    (col("sentiment").cast("integer") + 1) / 2
)

# Remove rows with missing values
df = df.dropna(subset=["text", "label"])

df.select("text", "sentiment", "label").show(5)


+--------------------+---------+-----+
|                text|sentiment|label|
+--------------------+---------+-----+
|Kickers on my wat...|        1|  1.0|
|user: AAP MOVIE. ...|        1|  1.0|
|user I'd be afrai...|        1|  1.0|
|   MNTA Over 12.00  |        1|  1.0|
|    OI  Over 21.37  |        1|  1.0|
+--------------------+---------+-----+
only showing top 5 rows



In [24]:
# chia train/test theo tỉ lệ  8:2
trainingData, testData = df.randomSplit(
    [0.8, 0.2],
    seed=42
)

print(f"Training samples: {trainingData.count()}")
print(f"Testing samples: {testData.count()}")


Training samples: 4682
Testing samples: 1109


In [25]:
# pipeline tiền xử lý
tokenizer = Tokenizer(
    inputCol="text",
    outputCol="words"
)

stopwordsRemover = StopWordsRemover(
    inputCol="words",
    outputCol="filtered_words"
)

hashingTF = HashingTF(
    inputCol="filtered_words",
    outputCol="raw_features",
    numFeatures=10000
)

idf = IDF(
    inputCol="raw_features",
    outputCol="features"
)


In [26]:
# mô hình Logistic Regression
lr = LogisticRegression(
    maxIter=10,
    regParam=0.001,
    featuresCol="features",
    labelCol="label"
)


In [27]:
# ghép các module trên thành 1 Spark ML Pipeline
pipeline = Pipeline(
    stages=[
        tokenizer,
        stopwordsRemover,
        hashingTF,
        idf,
        lr
    ]
)


In [28]:
# train mô hình
model = pipeline.fit(trainingData)


# dự đoán
predictions = model.transform(testData)

predictions.select(
    "text",
    "label",
    "prediction",
    "probability"
).show(10, truncate=80)


+--------------------------------------------------------------------------------+-----+----------+------------------------------------------+
|                                                                            text|label|prediction|                               probability|
+--------------------------------------------------------------------------------+-----+----------+------------------------------------------+
|  ISG An update to our Feb 20th video review..if it closes below 495 much low...|  0.0|       1.0|   [0.2614599786780246,0.7385400213219754]|
|  The rodeo clown sent BK screaming into the SI weekly red zone...time to pee...|  0.0|       0.0| [0.9999997991875168,2.008124831975877E-7]|
|                            , ES,SPY, Ground Hog Week, distribution at highs..  |  0.0|       1.0|   [0.0541093238048886,0.9458906761951114]|
|                                                        ES, S  PAT TWO, update  |  0.0|       0.0|[0.9986626113379365,0.0013373886620634545]|

In [29]:
# đánh giá mô hình
accuracy_evaluator = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="accuracy"
)

f1_evaluator = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="f1"
)

accuracy = accuracy_evaluator.evaluate(predictions)
f1 = f1_evaluator.evaluate(predictions)

print("=== Evaluation Results ===")
print(f"Accuracy : {accuracy:.4f}")
print(f"F1-score : {f1:.4f}")


=== Evaluation Results ===
Accuracy : 0.7295
F1-score : 0.7266


#### Nhận xét:
**Accuracy/F1 ~ 0.73** là hợp lý đối với mô hình baseline sử dụng
**TF-IDF kết hợp với Logistic Regression**.

Ở phần cải tiến (kỳ vọng cho ra kết quả tốt hơn), em thay thế Logistic Regression bằng **Naive Bayes** trong khi
giữ nguyên đặc trưng TF-IDF. Naive Bayes là mô hình xác suất đơn giản nhưng
thường cho hiệu quả tốt trong các bài toán phân loại văn bản, đặc biệt khi
đặc trưng đầu vào là các vector TF-IDF không âm.

In [30]:
# import và khai báo mô hình
from pyspark.ml.classification import NaiveBayes


nb = NaiveBayes(
    featuresCol="features",
    labelCol="label",
    smoothing=1.0,
    modelType="multinomial"
)



In [31]:
# pipeline Naive Bayes thay vì LR
pipeline_nb = Pipeline(
    stages=[
        tokenizer,
        stopwordsRemover,
        hashingTF,
        idf,
        nb
    ]
)


In [ ]:
# train
model_nb = pipeline_nb.fit(trainingData)

# dự đoán
predictions_nb = model_nb.transform(testData)

accuracy_nb = accuracy_evaluator.evaluate(predictions_nb)
f1_nb = f1_evaluator.evaluate(predictions_nb)

print("=== Improved Model (Naive Bayes) ===")
print(f"Accuracy : {accuracy_nb:.4f}")
print(f"F1-score : {f1_nb:.4f}")


=== Improved Model (Naive Bayes) ===
Accuracy : 0.6844
F1-score : 0.6842


#### Nhận xét:
- Mô hình baseline sử dụng **TF-IDF kết hợp với Logistic Regression** đạt
Accuracy/F1 khoảng **0.73**, cho thấy đây là một cấu hình ổn định đối với
bài toán phân loại cảm xúc.

- Trong phần cải tiến, em thay thế Logistic Regression bằng **Naive Bayes**
trong khi giữ nguyên đặc trưng TF-IDF. Tuy nhiên, kết quả thực nghiệm cho thấy
mô hình Naive Bayes không cải thiện hiệu năng so với baseline, thậm chí có
xu hướng giảm nhẹ.

- Nguyên nhân có thể do giả định độc lập có điều kiện của Naive Bayes chưa phù
hợp với dữ liệu thực tế, trong khi Logistic Regression có khả năng học các
trọng số đặc trưng linh hoạt hơn. Ngoài ra, kích thước tập dữ liệu tương đối
nhỏ cũng khiến Naive Bayes khó phát huy ưu thế của mình.

- Kết quả này cho thấy Logistic Regression vẫn là lựa chọn phù hợp hơn cho tập
dữ liệu hiện tại, đồng thời nhấn mạnh tầm quan trọng của việc lựa chọn mô hình
phù hợp với đặc điểm dữ liệu thay vì chỉ sử dụng các mô hình phức tạp hơn.
